https://pythonspeed.com/
Классный ресурс по оптимизации вычислений и работе с памятью

In [1]:
%load_ext autoreload
%autoreload 2
%aimport parallelism.multithreading

# Ускорение вычислений
Для того, чтобы проверять скорость вычислений в jupyter можно использовать либо %%time перед всем кодом в ячейке,
либо %time или %timeit перед строчкой выполнения кода

In [6]:
%%time
x = sum((i for i in range(100000)))

CPU times: user 5.42 ms, sys: 0 ns, total: 5.42 ms
Wall time: 5.37 ms


*Wall time* показывает суммарное время работы вашего кода.
*CPU times* показываем суммарное время, когда ваш процессор был занят.
При этом *sys* - это время, связанное с обращениями к операционной системе.

Если *wall time* превышает *CPU time*, то это значит, что программа ждет еще каких-то ответов, помимо вычислений процессора.

Таким образом, возможны следующие соотношения:

1. CPU time/ wall time  ≈ 1: Процесс тратит все время на использование CPU. Более быстрый CPU позволит ускорить данный процесс.
2. CPU time/ wall time < 1: Чем ниже отношение, тем больше CPU тратит на ожидание ответа (сети, жесткого диска и т.д., в том числе и просто режим сна).
Например, если он равен 0.75, то 25% времени CPU тратит на ожидание.

In [7]:
%time
sum([i for i in range(10000)])
%time
l = [a for a in range(10000)]

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 3.58 µs
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs


In [8]:
%timeit sum((i for i in range(10000)))
%timeit sum([i for i in range(10000)])

450 µs ± 6.49 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
274 µs ± 1.22 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Numpy

Данный пакет написан на С++ и позволяет совершать самые быстрые численные операции на всем диком Python.

In [19]:
import numpy as np
%timeit np.sum(np.array((i for i in range(10000))))
%timeit np.sum(np.array([i for i in range(10000)]))
%timeit sum((i for i in range(10000)))

5.72 µs ± 123 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
797 µs ± 22.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
456 µs ± 3.05 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [16]:
%timeit np.sum(np.fromiter((i for i in range(10)), dtype=int))
%timeit np.sum([i for i in range(10)])
%timeit sum((i for i in range(10)))

5 µs ± 23.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
5.74 µs ± 50.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
786 ns ± 7.35 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [17]:
# Индексация
a = np.array([[1,2,3,4], [5,6,7,8], [9,10,11,12]])
print(a)
print(a[0, 1])
print(a[:2, :3])
print(a[1:2, 2:3])
print(a[1:, 2:])
print(a[1, :])
print(a[1:2, :])
print(a[[1], :])
print(a[:, 1])
print(a[:, 1:2])

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]
2
[[1 2 3]
 [5 6 7]]
[[7]]
[[ 7  8]
 [11 12]]
[5 6 7 8]
[[5 6 7 8]]
[[5 6 7 8]]
[ 2  6 10]
[[ 2]
 [ 6]
 [10]]


In [28]:
# Размерность массивов
b = np.array([[1,2,3], [4,5,6], [7,8,9]])
print(b.shape)

# Изменение размерности (умноженные размерности совпадают)
v = np.array([1,2,3])
print(v, "\n", v.reshape(3, 1))

print(np.arange(16).reshape(2, 4, 2))
print(np.arange(16).reshape(1, 4, 2, 2))

# Изменение размерности (новая размерность другая - заполнение нулями)
a0 = np.arange(4)
a0.resize((8,))
print(a0)

(3, 3)
[1 2 3] 
 [[1]
 [2]
 [3]]
[[[ 0  1]
  [ 2  3]
  [ 4  5]
  [ 6  7]]

 [[ 8  9]
  [10 11]
  [12 13]
  [14 15]]]
[[[[ 0  1]
   [ 2  3]]

  [[ 4  5]
   [ 6  7]]

  [[ 8  9]
   [10 11]]

  [[12 13]
   [14 15]]]]
[0 1 2 3 0 0 0 0]


In [12]:
# Сортировка

c = np.array([[4, 3, 5], [1, 2, 1]])
print(np.sort(c, axis=1))

# Возвращает индексы элементов, которые были отсортированы
j = np.argsort(c)
print(j)
print(c[:, j])

[[3 4 5]
 [1 1 2]]
[[1 0 2]
 [0 2 1]]
[[[3 4 5]
  [4 5 3]]

 [[2 1 1]
  [1 1 2]]]


In [13]:
# Присвоение по условию (если, то)
data = np.random.randn(7, 4)
np.where(data, data>0, 0)

array([[1, 0, 0, 0],
       [1, 1, 1, 1],
       [0, 1, 1, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [1, 1, 1, 0],
       [1, 1, 0, 0]])

In [34]:
# Слияние массивов
x,y,z = np.arange(1,3), np.arange(3,5), np.arange(5,7)
print(np.concatenate([x,y,z]))

array2D_1 = np.array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])
array2D_2 = np.array([[10, 11, 12],
       [13, 14, 15],
       [16, 17, 18]])

# Вертикальное слияние
print(f"Вертикальное : \n {np.vstack((array2D_1, array2D_2))}")

# Горизонтальное слияние
print(f"Горизонтальное : \n {np.hstack((array2D_1, array2D_2))}")

[1 2 3 4 5 6]
Вертикальное : 
 [[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [10 11 12]
 [13 14 15]
 [16 17 18]]
Горизонтальное : 
 [[ 0  1  2 10 11 12]
 [ 3  4  5 13 14 15]
 [ 6  7  8 16 17 18]]


In [38]:
# Применение функции к разным осям массива
m = np.random.randn(3,2)
# Сначала указываем функцию, потом по какой оси, а затем указываем сам массив
print(np.apply_along_axis(lambda x: sum(x ** 2), 1, m))
print(np.apply_along_axis(lambda x: sum(x ** 2), 0, m))
# Обратим внимание, что у функции может быть только один аргумент - это значения массива из конкретной оси

[0.31300339 0.07865637 0.67738013]
[0.58474968 0.4842902 ]


In [41]:
# Траблы с округлением при сравнении
print(np.round(0.3, 15) == np.round(3 * 0.1, 15))
print(np.round(0.3, 16) == np.round(3 * 0.1, 16))
print(np.round(0.3, 17) == np.round(3 * 0.1, 17))
# Используем такую функцию, чтобы проверить, равны ли массивы
print(np.allclose(0.3, 3*0.1))

True
True
False
True


# Работа с памятью

Помним, что у любого компьютера есть ограничение по оперативной памяти, а все наши рабочие процессы хранятся именно там.
Любой сайт - это такой же компьютер, как и у вас, со своим жестким диском и оперативной памятью.
Таким образом, помимо скорости вычислений, нам бы хотелось еще и не вылазить за пределы оперативной памяти.
В противном случае, программа просто прекратит свою работу и придется запускать все заново.


In [6]:
import sys
data = np.random.randn(7, 4, 10000)
sys.getsizeof(data)  # Данная команда показывает, сколько байтов содержит датасет

2240136

In [4]:
sys.getsizeof(data) / 2 ** 20

2.1363601684570312

## Garbage collector

https://stackify.com/python-garbage-collection/

В Python, в отличие от более ранних языков программирования, не нужно самим напрямую управлять памятью.
Здесь реализовано автоматическое управление памятью.
Очевидный плюс для разработчика - можно быстрей писать программы, не тратя время на выделение памяти для каждой переменной.
Очевидный минус - если мы совсем перестаем контроллировать этот процесс, то может выйти за пределы оперативной памяти.

Существует два основных аспекта garbage collection в CPython:

- Reference counting
- Generational garbage collection

### Reference counting garbage collection

Где бы не создавался питоновский объект, базовый С объект использует питоновский тип (например, лист, словарь или функцию) и количество отсылок.
Чем чаще мы отсылаемся к какому-то объекту, тем больше накапливается этого показателя.
Основные способы сослаться на какой-либо объект:
- присвоить объект переменной
- добавить объект к структуре данных (например, добавление в список или как *property* в классе)
- вставить объект как аргумент функции

In [23]:
c1 = "new_var"
sys.getrefcount(c1)

2

In [14]:
a = 'my-string'
sys.getrefcount(a)
# Всего было 3 отсылки к объекту a - создание раннее, переприсваивание сейчас и вызов в функции sys.getrefcount

3

In [24]:
b = [a] # Make a list with a as an element.
c = { 'key': a } # Create a dictionary with a as one of the values.
sys.getrefcount(a)

4

### Generational garbage collection



In [25]:
class MyClass(object):
       pass
a = MyClass()
a.obj = a
del a
a

NameError: name 'a' is not defined

Здесь мы определили новый класс.
Затем создали экземпляр класса и присвоили экземпляру аттрибут в качестве самого класса. В итоге мы удалили объект.
Несмотря на то, что объект мы уже удалили, в Python все еще хранится экземпляр в памяти.
Такую проблему мы называем исходный цикл (reference cycle). Эта и многие другие проблемы решаются при помощи модуля *gc*.

Существуют два основных понятия для garbage collection:
- поколение (generation)
- порог (threshold)

Garbage collector отслеживает абсолютно все объекты, которые когда-либо попадали в Python.
Всего существует 3 поколения для объектов и объекты в зависимости от наших действий передвигаются от старого к более новому.
Для каждого поколения *gc* модуль выделены пороги для числа объектов. Если число объектов переваливает за этот порог,
то *gc* запустит процесс сбора. Любые объекты, которые переживут этот процесс, попадут в более старое поколение.

В отличие от проблемы с отсылками, здесь мы можем управлять этим процессом.
Во-первых, мы можем изменять пороги, которые влияют на вызов процесса сборки мусора.
Во-вторых, можем вручную вызывать сборщик мусора или, наоборот, отключать его.

In [26]:
import gc
print(gc.get_threshold())
print(gc.get_count())

(700, 10, 10)
(134, 5, 2)


In [27]:
gc.collect()

10

In [28]:
gc.get_count()

(246, 0, 0)

In [38]:
# Изменим порог для перехода в другие поколения
gc.set_threshold(1000, 15, 15)
gc.get_threshold()

(1000, 15, 15)

Увеличение порога сократит частоту запуска сборщика мусора.
Это ускорит наши процессы, но все объекты будут дольше храниться в памяти.

## Типы данных

Вспомним, что Python - динамический язык программирования. Однако, для более ручного управления памятью и кодом, мы можем объектам присваивать
типы данных самостоятельно. Рекомендуется использовать типы данных *numpy* массивов.

1. int - целочисленный тип данных: (-2, -1, 0, 1, ....)
2. uint - целочисленный тип данных: (0, 1, 2, .....)
3. float - вещественный тип данных с плавающей точкой
4. bool - логический тип данных
5. string - строковый тип данных

После int, uint, float мы можем указать числа, которые будут отвечать за длину диапазона возможных значений.
Например, np.int32 означает, что диапазон значений включает в себя 2 ** 32 (около 4 млрд) чисел, которые расположены симметрично относительно нуля;
np.uint8 - диапазон чисел от 0 до 255

Отметим, что в случае np.float32, np.float64 активируются Ctype типы данных. Это значит, что если нам нужна скорость обработки, то
мы должны использовать именно эти типа данных для вещественных чисел. В противном случае, можно использовать float16


In [37]:
print(np.uint8(255))
print(np.uint8(1000))  # 256 + 256 + 256 + 232
print(np.float64(5/3))
print(np.float16(5/3))

255
232
1.6666666666666667
1.667


Давайте позапускаем разные коды и посмотрим, сколько уходит памяти в каждой строчке.

In [23]:
import resource

usage = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss  # выдает значения в Кб
print(usage)

99016


In [25]:
from memory_checking_1 import make_big_array, make_two_arrays
ar = make_big_array()
print(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss)

99016


In [50]:
# Peak memory usage
ar1, ar2 = make_two_arrays()
print(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 2 ** 10)

177.78125


In [51]:
print(sys.getsizeof(ar1) / 2 ** 20)
print(sys.getsizeof(ar2) / 2 ** 20)

80.00012969970703
80.00012969970703


In [27]:
import tracemalloc
tracemalloc.start()
make_two_arrays()
current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / (1024 * 1024)}MB; Peak was {peak / (1024 * 1024)}MB")
tracemalloc.stop()

Current memory usage is 0.0008916854858398438MB; Peak was 160.00118160247803MB


In [28]:
def memory_analyze(func, result="Mb", *args):
    tracemalloc.start()
    func(*args)
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    if result == "Mb":
        print(f"Current memory usage is {current / (1024 * 1024)}MB; Peak was {peak / (1024 * 1024)}MB")
    elif result == "Gb":
         print(f"Current memory usage is {current / (1024 * 1024 * 1024)}Gb; Peak was {peak / (1024 * 1024 * 1024)}Gb")
    elif result == "Kb":
        print(f"Current memory usage is {current / 1024}KB; Peak was {peak / 1024}KB")
    else:
        ValueError("Insert correct type size")

    return
memory_analyze(make_big_array, result="Gb")

Current memory usage is 4.231929779052734e-06Gb; Peak was 0.3906293138861656Gb


## Учимся экономить память

### Проблемы с локальными переменными в функции

In [29]:
def load_1GB_of_data():
    return np.ones((2 ** 30), dtype=np.uint8)

def modify1(data):
    return data * 2

def modify2(data):
    return data + 10

def process_data():
    data = load_1GB_of_data()
    return modify2(modify1(data))

memory_analyze(process_data, "Gb")

Current memory usage is 1.5730038285255432e-06Gb; Peak was 3.000001926906407Gb


Как так вышло? Мы создали данные объемом 1 Гб. Два раза умножили на 2 и прибавили и объем наших данных увеличился в 3 раза.
По-хорошему, объем должен был вырасти до 2 Гб. Однако, откуда здесь еще лишний 1 Гб?
Весь прикол как раз в сборщике мусора, который учел количество обращений в объекту не 2, а 3 раза!
Есть 3 способа борьбы с этим

In [30]:
# 1. Не использовать локальную переменную
def process_data():
    return modify2(modify1(load_1GB_of_data()))
memory_analyze(process_data, "Gb")

Current memory usage is 1.1296942830085754e-06Gb; Peak was 2.000001401640475Gb


In [31]:
# 2. Переприсвоить локальной переменной
def process_data():
    data = load_1GB_of_data()
    data = modify1(data)
    data = modify2(data)
    return data
memory_analyze(process_data, "Gb")

Current memory usage is 1.5040859580039978e-06Gb; Peak was 2.0000017499551177Gb


In [ ]:
# 3. Передача права собственности на объект
class Owner:
    def __init__(self, data):
        self.data = data

def process_data():
    data = Owner(load_1GB_of_data())
    return modify2(modify1(data))

def modify1(owned_data):
    data = owned_data.data
    # Remove a reference to original data:
    owned_data.data = None
    return data * 2

### Излишняя модификация и присваивание

In [32]:
def normalize(array: np.ndarray) -> np.ndarray:
    """
    Takes a floating point array.

    Returns a normalized array with values between 0 and 1.
    """
    low = array.min()
    high = array.max()
    return (array - low) * (high - low)
memory_analyze(normalize, "Kb", np.random.randint(1, 10, 10000).reshape(10, 100, 10))

Current memory usage is 1.8056640625KB; Peak was 158.5322265625KB


In [33]:
# In-place type of assigning
def normalize_in_place(array: np.ndarray):
    low = array.min()
    high = array.max()
    result = array.copy()  # Это нужно для того, чтобы не менять значения исходного массива!
    array -= low
    array *= high - low
    return array
memory_analyze(normalize_in_place, "Kb", np.random.randint(1, 10, 10000).reshape(10, 100, 10))

Current memory usage is 0.953125KB; Peak was 79.478515625KB


In [34]:
import numpy as np
from memory_profiler import memory_usage
interval_sec = 0.2
memory_changing = memory_usage((make_two_arrays), interval=interval_sec)
dict(zip(np.cumsum([interval_sec] * len(memory_changing)),
         memory_changing))

{0.2: 178.046875,
 0.4: 178.11328125,
 0.6000000000000001: 192.11328125,
 0.8: 202.11328125,
 1.0: 214.11328125,
 1.2: 228.11328125,
 1.4: 242.11328125,
 1.5999999999999999: 257.5078125,
 1.7999999999999998: 178.046875}

Давайте сравним расходы памяти в случае генератора и списка.

In [35]:
from memory_checking_1 import (stack_using_list, stack_using_generator)
n_images = 500
n_pixels = 1000
snapshot_interval = 0.2 # seconds
list_mem_usage = memory_usage((stack_using_list,(n_images, n_pixels)), interval=snapshot_interval)
gen_mem_usage = memory_usage((stack_using_generator,(n_images, n_pixels)), interval=snapshot_interval)

In [36]:
dict(zip(np.cumsum([snapshot_interval] * len(list_mem_usage)),
         list_mem_usage))

{0.2: 98.45703125,
 0.4: 98.58203125,
 0.6000000000000001: 303.3125,
 0.8: 526.01171875,
 1.0: 684.73828125,
 1.2: 907.14453125,
 1.4: 1158.9765625,
 1.5999999999999999: 1326.78125,
 1.7999999999999998: 1558.0703125,
 1.9999999999999998: 1820.9453125,
 2.1999999999999997: 2080.06640625,
 2.4: 2342.73046875,
 2.6: 2600.1328125,
 2.8000000000000003: 2861.07421875,
 3.0000000000000004: 3123.48828125,
 3.2000000000000006: 3288.515625,
 3.400000000000001: 3450.91796875,
 3.600000000000001: 3601.13671875,
 3.800000000000001: 3743.86328125,
 4.000000000000001: 3922.58984375,
 4.200000000000001: 3922.58984375,
 4.400000000000001: 3922.58984375,
 4.600000000000001: 3922.58984375,
 4.800000000000002: 98.62109375}

In [37]:
dict(zip(np.cumsum([snapshot_interval] * len(gen_mem_usage)),
         gen_mem_usage))

{0.2: 98.62109375,
 0.4: 98.640625,
 0.6000000000000001: 120.953125,
 0.8: 120.953125,
 1.0: 120.953125,
 1.2: 120.953125,
 1.4: 120.953125,
 1.5999999999999999: 120.953125,
 1.7999999999999998: 120.953125,
 1.9999999999999998: 120.953125,
 2.1999999999999997: 120.953125,
 2.4: 120.953125,
 2.6: 120.953125,
 2.8000000000000003: 120.953125,
 3.0000000000000004: 120.953125,
 3.2000000000000006: 120.953125,
 3.400000000000001: 120.953125,
 3.600000000000001: 106.0625}

# Параллелизация: мультипроцессинг и многопоточность

https://realpython.com/python-concurrency/

Все, что мы делали до этого, мы делали на 1 ядре процессоре. Но как мы знаем, у каждого современного процессора больше 1 ядра, а у каждого ядра есть два потока!
Соответственно, мы можем выполнять идентичные друг другу процессы параллельно!
Однако, давайте сначала рассмотрим основные виды программ.

***CPU Bound*** означает, что скорость выполнения процесса ограничена скоростью центрального процессора. Задача, выполняющая вычисления на небольшом наборе чисел, например, перемножение небольших матриц, скорее всего, будет привязана к процессору.

***I/O Bound*** означает, что скорость выполнения процесса ограничена скоростью подсистемы ввода-вывода. Задача, которая обрабатывает данные с диска, например, подсчитывает количество строк в файле, скорее всего, будет связана с вводом-выводом.

***Memory bound*** означает, что скорость выполнения процесса ограничена объемом доступной оперативной памяти и скоростью доступа к ней. Задача, которая обрабатывает большие объемы данных в памяти, например, перемножение больших матриц, скорее всего, будет ограничена памятью.

***Cache bound*** означает, что скорость выполнения процесса ограничена объемом и скоростью доступного кэша. Задача, которая просто обрабатывает больше данных, чем помещается в кэш, будет ограничена кэшем.


Теперь рассмотрим способы ускорения программ.

***Параллелизм (parallelism)*** - одновременное выполнение нескольких операций.

***Поточность (threading)*** - это модель параллельного выполнения, при которой несколько потоков поочередно выполняют задачи. Один процесс может содержать несколько потоков. У Python сложные отношения с потоками благодаря его GIL, но эти подробности мы разбирать не будем. Главное запомнить, что поточность используется, когда нам нужно ускорить I/O задачи (задачи, связанные с импортом и выводом данных).
Нативный Python использует библиотеку **threading**

***Мультипроцессинг (multiprocessing)*** - это один из способов для осуществления параллелизма, которое предполагает распределение задач по центральным процессорам (CPU, или ядрам) компьютера. Многопроцессорная обработка хорошо подходит для задач, привязанных к процессору: в эту категорию обычно попадают жестко привязанные циклы и математические вычисления.
Нативный Python использует библиотеку **multiprocessing**

***Одновременность / многопоточность (concurrency)*** - это несколько более широкий термин, чем параллелизм. Он подразумевает, что несколько задач могут выполняться параллельно. (Существует поговорка, что одновременность не подразумевает параллельности).
Нативный Python использует библиотеку **concurrent.futures**

Обычную поточность мы тут рассматривать не будем. Давайте посмотрим, как реализуется одновременная многопоточность.

## Многопоточность

In [47]:
%%time
# Один поток
import requests
from parallelism.multithreading import urls
results = []
for url in urls:
    with requests.get(url) as src:
        results.append(src.content)

CPU times: user 242 ms, sys: 18 ms, total: 260 ms
Wall time: 2.25 s


In [66]:
%%time
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor(8) as executor:  # 4 потока (2 ядра по 2 потока)
    results = executor.map(requests.get, urls)
results = list(map(lambda x: x.content, results))

CPU times: user 147 ms, sys: 14.6 ms, total: 162 ms
Wall time: 313 ms


## Мультипроцессинг

In [68]:
%%time
from multiprocessing import Pool
from parallelism.multiprocessing import if_prime

# answer = 0
#
# for i in range(1000000):
#     answer += if_prime(i)
answer = sum(map(lambda x: if_prime(x), range(int(1e6))))

CPU times: user 3.95 s, sys: 61.4 ms, total: 4.01 s
Wall time: 4.02 s


In [91]:
%%time
# with Pool(4) as p:  # 4 потока на 2 ядрах
#     answer = sum(p.map(if_prime, range(int(1e6))))
with Pool(4) as p:  # 4 потока на 2 ядрах
    answer = p.map(if_prime, range(int(1e6)))

CPU times: user 1.59 s, sys: 235 ms, total: 1.83 s
Wall time: 1.79 s


Давайте попробуем применить многопоточность на численных операциях и мультипроцессинг на операциях I/O.

In [77]:
%%time
with ThreadPoolExecutor(4) as executor:  # 4 потока (2 ядра по 2 потока)
    results = executor.map(if_prime, range(int(1e6)))

CPU times: user 39.5 s, sys: 9.11 s, total: 48.6 s
Wall time: 41.4 s


In [82]:
%%time
with Pool(8) as p:  # 4 потока на 2 ядрах
    answer = p.map(requests.get, urls)
answer = list(map(lambda x: x.content, answer))

CPU times: user 19.2 ms, sys: 249 ms, total: 268 ms
Wall time: 617 ms


Как видим, многопоточность сильно хуже справилась с численными операциями, а мультипроцессинг - с операциями ввода-вывода.

## Joblib


In [87]:
%%time
from joblib import Parallel, delayed
results = sum(Parallel(n_jobs=4, verbose=3)(delayed(if_prime)(i) for i in range(int(1e6))))  # n_jobs = -1 - все потоки

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  44 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 196604 tasks      | elapsed:    1.6s


CPU times: user 4.08 s, sys: 140 ms, total: 4.22 s
Wall time: 4.52 s


[Parallel(n_jobs=4)]: Done 1000000 out of 1000000 | elapsed:    4.5s finished


In [88]:
%%time
results = Parallel(n_jobs=-1, verbose=3)(delayed(requests.get)(url) for url in urls)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of  13 | elapsed:    0.7s remaining:    4.0s
[Parallel(n_jobs=-1)]: Done   7 out of  13 | elapsed:    0.9s remaining:    0.8s


CPU times: user 25.8 ms, sys: 347 ms, total: 373 ms
Wall time: 1.01 s


[Parallel(n_jobs=-1)]: Done  13 out of  13 | elapsed:    0.9s finished


## p_tqdm


In [95]:
%%time
from p_tqdm import p_map
results = p_map(if_prime, range(int(1e6)), num_cpus=-1)

  0%|          | 0/1000000 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Ассинхронность

https://realpython.com/async-io-python/


Пакет **asyncio** в документации Python называется библиотекой для написания параллельного кода. Однако **async IO** - это не многопоточность и не многопроцессорность.
На самом деле, **async IO** - это однопоточная, однопроцессная конструкция: в ней используется ***кооперативная многозадачность***. Другими словами, **async IO** дает ощущение параллелизма, несмотря на использование одного потока в одном процессе.

***Корутины*** (coroutines, центральная особенность async IO) можно планировать параллельно, но они не являются параллельными по своей природе.

Важно запомнить, что асинхронные программы выполняются на 1 потоке.

Асинхронное программирование применяется для задач I/O. Зачастую это задачи, связанные с передачей данных по сети Интернет. Как мы выяснили, для этой задачи наиболее подходит многопоточность (concurrent.futures). Давайте проверим это еще на одном примере.

In [7]:
from parallelism.multithreading import download_all_sites
import time
sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
      ] * 80
start_time = time.time()
download_all_sites(sites)
duration = time.time() - start_time
print(f"Downloaded {len(sites)} in {duration} seconds")

Read 10490 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10490 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10490 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10490 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10490 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10490 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10490 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10490 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10490 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10490 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10490 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10490 from https://www.jyth

In [8]:
from parallelism.multithreading import download_all_sites_concur
start_time = time.time()
download_all_sites_concur(sites)
duration = time.time() - start_time
print(f"Downloaded {len(sites)} in {duration} seconds")

Downloaded 160 in 0.014427900314331055 seconds


![widht=500](https://files.realpython.com/media/Threading.3eef48da829e.png)

Одна из главных проблем многопоточности - сложно уловить, в какой последовательности работает каждый поток.

Общая концепция asyncio заключается в следующем. ***Event loop*** управляет тем, как и когда выполняется каждая задача.
***Event loop*** знает о каждой задаче и знает, в каком состоянии она находится, но 